In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_orderbooks_from_file, get_binance_orderbook, get_omnipool, load_omnipool, save_omnipool, load_config
from hydradx.model.processing import get_omnipool_data_from_file, get_centralized_market
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent_general as arb
from hydradx.model.amm.omnipool_router import OmnipoolRouter

from pprint import pprint

import json

Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [6]:
# load Omnipool
print('loading omnipool...')

if get_live_markets:
    omnipool = get_omnipool()
else:
    omnipool = load_omnipool()

fourpool, btcpool, twopool = omnipool.sub_pools.values()
router = OmnipoolRouter(
    exchanges=[omnipool, twopool, fourpool, btcpool]
)

loading omnipool...


In [7]:
# load centralized markets
print('loading centralized markets...')
# arb_file = "arbconfig.txt"
cfg = load_config("arbconfig5.txt", path="config")

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}

cex_dict = {}
for exchange in ('kraken', 'binance'):
    if get_live_markets:
        cex_dict[exchange] = get_centralized_market(
            config=cfg, 
            exchange_name=exchange, 
            trade_fee=cex_fees[exchange],
            archive=False
        )
    else:
        cex_dict[exchange] = CentralizedMarket(
            order_book=get_orderbooks_from_file("archive/")[exchange],
            unique_id=exchange,
            trade_fee=cex_fees[exchange]
        )
print("done.")

loading centralized markets...
done.


In [10]:
print('calculating arbitrage...')
binance = cex_dict['binance']
kraken = cex_dict['kraken']
exchanges = {'router': router, 'binance': binance, 'kraken': kraken}
all_swaps = arb.get_arb_swaps(exchanges, cfg)
for swap in all_swaps:
    print(swap)

calculating arbitrage...
{'router': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'iBTC', 'amount': 18.48, 'max_sell': 0.0022417830915384086}, 'binance': {'trade': 'sell', 'buy_asset': 'BTC', 'sell_asset': 'DOT', 'amount': 18.48, 'price': 0.0001237381}}
{'router': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'iBTC', 'amount': 698.59, 'max_sell': 0.08508700926578065}, 'binance': {'trade': 'sell', 'buy_asset': 'BTC', 'sell_asset': 'DOT', 'amount': 698.59, 'price': 0.00012363815}}
{'router': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'iBTC', 'amount': 2.7423583632812507, 'max_sell': 0.13365613634635115}, 'binance': {'trade': 'sell', 'buy_asset': 'BTC', 'sell_asset': 'ETH', 'amount': 2.7423583632812507, 'price': 0.049085445000000005}}
{'router': {'trade': 'buy', 'buy_asset': 'GLMR', 'sell_asset': 'USDT10', 'amount': 574.9, 'max_sell': 239.87555272104862}, 'binance': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'GLMR', 'amount': 574.9, 'price': 0.4205896}}


In [11]:
test_ex = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
init_agent = Agent(
    holdings={tkn: 10000000 for tkn in [tkn for ex in [omnipool, twopool, fourpool, btcpool, kraken, binance] for tkn in ex.asset_list]}
)
test_agent = init_agent.copy()
arb.execute_arb(test_ex, test_agent, all_swaps)

equivalency_map = {
    'XETH': 'ETH',
    'WETH': 'ETH',
    'WBTC': 'BTC',
    'iBTC': 'BTC',
    'XBT': 'BTC',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'USDT23': 'USD',
    'USDT10': 'USD',
    'USDC22': 'USD',
    'USDC21': 'USD',
    'DAI': 'USD',
}

profit = arb.calculate_profit(init_agent, test_agent, equivalency_map)
print("profit in USD as Binance spot prices:", exchanges['binance'].value_assets(profit, equivalency_map))

# # try rerouting to see if we can squeeze some more profit out of it
# agent3 = init_agent.copy()
# state3 = init_state.copy()
# cex_dict3 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
# all_swaps3 = arb.combine_swaps(
#     dex=state3,
#     cex=cex_dict3,
#     agent=agent3,
#     all_swaps=all_swaps2,
#     asset_map=asset_map
# )
# arb.execute_arb(state3, cex_dict3, agent3, all_swaps3)
# profit3 = arb.calculate_profit(init_agent, agent3, asset_map)

# print("strategy 3 profit")
# print(profit3)
# print("strategy 3 profit at Binance spot")
# print(cex_dict3['binance'].value_assets(profit3, asset_map))


profit in USD as Binance spot prices: 156.25934903734785
